# Practical Session Appendix

## Reference Guide

### Objective 

Python libraries are Python files that contain definitions of functions and objects. These functions and objects can be used within your code by importing them with the `import` keyword. There are a large number of available python libraries that you can use. Some of the standard python libraries (that come with Python by default) that you should consider learning are `sys` (system), `os` (operating system) and `re` (regular expressions). We have written libraries that we will use during this course.This guide aims to introduce some (not all) functions and classes from the libraries.

### Outline
* Example classes Sequence and Alphabet
* Files
    * DNA and protein sequence data
    * Sequence alignments
    * Phylogenetic trees
    * Genome sequence: 2-bit files
    * Browser Extensible Data (BED) files
    * Gene expression



---

## Example classes Sequence and Alphabet ##

Previously we have represented biological sequences as text strings and we will keep doing that. However, biological sequences are made up of much fewer symbols than there are characters. When we analyse sequences we need to define all possible symbols that can occur, so we can find errors, count them and define “sequence patterns” etc. 

So we will define an `Alphabet` class. This will be used to define the `Sequence` class, which will have methods for counting and finding symbols. All of that will be made permanent by means of modules. Then we can truly start doing serious work with sequences. A reduced version of the `Sequence` and `Alphabet` classes are shown below.

In [1]:
class Alphabet():
    """ A minimal class for alphabets """
    def __init__(self, symbolString):
        self.symbols = symbolString
    def __len__(self):              # implements the "len" operator
        return len(self.symbols)
    def __contains__(self, sym):    # implements the "in" operator
        return sym in self.symbols
""" Below we declare alphabets that are going to be available when 
this module is imported """ 
DNA_Alphabet = Alphabet('ACGT')
RNA_Alphabet = Alphabet('ACGU')
Protein_Alphabet = Alphabet('ACDEFGHIKLMNPQRSTVWY')

class Sequence():
    """ A biological sequence class. Stores the sequence itself, 
        the alphabet and a name. """
    def __init__(self, sequence, alphabet, name):
        for sym in sequence:
            if not sym in alphabet:  # error check: bail out
                raise RuntimeError('Invalid symbol: ' + sym)
        self.sequence = sequence
        self.alphabet = alphabet
        self.name = name
    def __len__(self):      # the "len" operator
        return len(self.sequence)
    def writeFasta(self):
        """ Write one sequence in FASTA format to a string and return it """
        fasta = '>' + self.name + '\n'
        data = self.sequence
        nlines = (len(self.sequence) - 1) // 60 + 1
        for i in range(nlines):
            lineofseq = ''.join(data[i*60 : (i+1)*60]) + '\n'
            fasta += lineofseq
        return fasta
    def __str__(self):
        return self.writeFasta()
    def getcount(self, findme):
        """ Get the number of occurrences of specified symbol """
        cnt = 0
        for sym in self.sequence:
            if findme == sym:
                cnt = cnt + 1
        return cnt
    def find(self, findme):
        """ Find the position of the symbol or sub-sequence """
        return self.sequence.find(findme)

These classes are not specific to DNA, RNA or proteins. They can be used for virtually any size alphabet or any length of sequence. The `__init__` function ensures that any alphabet or sequence is properly initialised before they are used: an alphabet is necessarily defined in terms of a list of symbols; a sequence always has a name, an alphabet and a sequence that is checked so that it does not have any symbols not defined by the alphabet. (Note we create three variables that correspond to these in the module.)

There are many so-called operators in Python, e.g. `==`, `in` and `len(obj` that are defined over objects from classes. Above, we define the len operator for both the `Alphabet` and `Sequence` class by implementing the function `__len__`. The `in` operator that is used on the `Alphabet` class in the `Sequence` `__init__` function (if not sym in alphabet) is defined by implementing the `__contains__` function (see the `Alphabet` class).

To see how we can use them.

In [2]:
myseq = Sequence('AGCCCGGAT', DNA_Alphabet, 'MyDNA')
myseq.getcount('G')

3

In [3]:
myseq.find('CGG')

4

---

## Files

Files are used to permanently store data. Many files but not all are text files, meaning that it is possible to view and edit them using a text editor, e.g. TextWrangler on Macs, Notepad on Windows PCs and emacs on Linux machines. To increase portability and readability, text files are often used for storing biological data. The information in them usually follows some pre-defined template or format. For example, biological sequence data is often stored in FASTA files (use a `.fa` or `.fasta` file extension).

Below we illustrate a couple of file formats, but these should only be seen as examples and you may not want to look at the code to carefully.

### DNA and protein sequence data
FASTA is by far the most popular file format for biological sequence data. A FASTA file contains a so-called defline that starts with a `>` then the name of the sequence followed on the same line by other information. The lines before the next `>` or `end-of-file` is the sequence data. The symbols used depend on what type of sequence. The following code consists of two methods where one calls the other and read a file on the FASTA format. It creates a sequence object for each entry in a specified file. (The definition of the used Sequence class is provided in the previous section of this guide.)

In [4]:
def readFastaString(string, alphabet):
    """ Read the given string as FASTA formatted data and return 
        the list of sequences contained within it. """
    seqlist = []    # list of sequences contained in the string 
    seqname = None  # name of *current* sequence 
    seqdata = []    # sequence data for *current* sequence
    for line in string.splitlines():    # read every line
        if len(line) == 0:              # ignore empty lines
            continue
        if line[0] == '>':  # start of new sequence            
            if seqname:     # check if we've got one current
                current = Sequence(seqdata, alphabet, seqname)
                seqlist.append(current)
            # now collect data about the new sequence
            seqname = line[1:].split()[0] # skip first char
            seqdata = []
        else: # we assume this is (more) data for current
            cleanline = line.split()
            for thisline in cleanline:
                seqdata.extend(tuple(thisline.strip('*')))
    # we're done reading the file, but the last sequence remains
    if seqname:
        lastseq = Sequence(seqdata, alphabet, seqname)
        seqlist.append(lastseq)
    return seqlist

def readFastaFile(filename, alphabet):
    """ Read the given FASTA formatted file and return the list 
        of sequences contained within it. """
    fh = open(filename)
    data = fh.read()
    fh.close()
    seqlist = readFastaString(data, alphabet)
    return seqlist

### Sequence alignment

Aligned sequences are organised in a logical way that can help identify conserved regions and, by comparing similarity of sequence, can tell us which sequences are closely or more distantly related. Multiple sequence alignments (MSAs) form the basis of much of phylogenetic analysis. "Good" sequence alignments can indicate evolutionary relationships between sequences at the level of individual positions, i.e. that they originate from the same ancestral sequence. Insertions and deletions are indicated through the presence of gaps. Alignments are used for different types of sequence (e.g. DNA or protein).

As programmers we need to create suitable data structures for the represented data, so that methods can operate on them effectively. `guide.py` contains a class of specific interest: `Alignment`.

There are several functions for reading and writing alignments to files. `.aln` files follow the so-called `Clustal` format and contain collections of aligned sequences. (You can use the FASTA format to store sequence alignments by padding the sequences with a gap character `-`, ensuring that all sequences are of the same length.)

In [5]:
def readClustalString(string, alphabet):
    """ Read a ClustalW2 alignment in the given string and return as an Alignment object. """
    seqs = {} # sequence data
    for line in string.splitlines():
        if line.startswith('CLUSTAL') or line.startswith('STOCKHOLM') or line.startswith('#'):
            continue
        if len(line.strip()) == 0:
            continue
        if line[0] == ' ' or '*' in line or ':' in line:
            continue
        sections = line.split()
        name, seq = sections[0:2]
        if seqs.has_key(name):
            seqs[name] += seq
        else:
            seqs[name] = seq
    sequences = []
    for name, seq in seqs.items():
        sequences.append(Sequence(seq, alphabet, name, gappy = True))
    return Alignment(sequences)

def readClustalFile(filename, alphabet):
    """ Read a ClustalW2 alignment file and return an Alignment object
    containing the alignment. """
    fh = open(filename)
    data = fh.read()
    fh.close()
    aln = readClustalString(data, alphabet)
    return aln

### Phylogenetic tree

Phylogenetic trees are used to represent and visualise the ancestral relationship between multiple genotypes. The Newick (or New Hampshire) tree format is a way of writing a tree in a simple text format. Each split in the tree is written as the two ends of the split in brackets, separated by a comma.

For example, assume you have four genotypes, `A`, `B`, `C` and `D`. You write that `A` is linked to `B` with `(A, B)`, and that `C` is linked to `D` as `(C, D)`; then these two pairs are linked together is written as `((A, B), (C, D))`. And it works for arbitrarily large, and potentially imbalanced trees.

There are methods to both read and write Newick files in `guide.py` but you need to inspect the Python code if you want to understand them.

### Genome sequence: 2-bit files

A 2-bit file is a compact binary file that contain the DNA sequence for a whole genome, divided into chromosomes. You address specific parts of the genome by chromosome name (e.g. "chr22") and a start and end location (e.g. 1000000 and 1000100). There is currently no provision for them in guide.py but you'll find it in `seqdata.py`.

### Browser Extensible Data: BED files

Available in `seqdata.py`.

### Gene expression

Another common file format in bioinformatics is for gene expression data: the GEO/SOFT format. It is designed to capture information produced by microarrays, intensity levels of many thousands of probes (corresponding to sections of DNA). A line in the main section of a file first contains a probe identifier, a gene name followed by (on the same line) a possibly large number of numeric values (intensities, possibly processed in some way).

In the beginning of this file there is also information about what those values mean, and the following code designed to read such files disregard most of this sometimes-important information. The following function will read a GEO file and create a dictionary with an entry for each probe/gene name (key) and all numeric values.

In [6]:
class GeneProfile():
    """ A class for gene expression data.
    Example usage:
    >>> gp = GeneProfile('MyMicroarray', ['Exp1', 'Exp2'])
    >>> gp['gene1'] = [0.1, 0.5]
    >>> gp['gene2'] = [2, 1]
    >>> gp.getSample('Exp2')
    {'gene1': [0.5], 'gene2': [1.0]}
    """
    def __init__(self, dataset_name='', sample_names=[], profiles = None):
        """ Create a gene profile set. """
        self.name = dataset_name
        self.samples = sample_names
        self.genes = profiles or {} # dictionary for storing all gene--measurement pairs
    def __setitem__(self, name, probevalues):
        if len(probevalues) == len(self.samples):
            self.genes[name] = [float(y) for y in probevalues]
        else:
            raise RuntimeError('Invalid number of measurements for probe ' + name)
    def __getitem__(self, name):
        return self.genes[name]
    def getSorted(self, index, descending=True):
        """Get a list of (gene, value) tuples in descending order by value"""
        key_fn = lambda v: v[1][index]
        return sorted(list(self.genes.items()), key=key_fn, reverse=descending)
    def getSample(self, sample_name):
        """Construct a gene dictionary including only named samples. """
        mygenes = {}
        if isinstance(sample_name, str):    # a single sample-name
            mysamples = [sample_name]
        else:                               # a list of sample-names
            mysamples = sample_name
        for gene in self.genes:
            mygenes[gene] = []
            for name in mysamples:
                mygenes[gene].append(self.genes[gene][self.samples.index(name)])
        return mygenes
    def __str__(self):
        """ Display data as a truncated GEO SOFT file named filename. """
        line = '^DATASET = ' + self.dataset + '\n'
        line += '!dataset_table_begin\nID_REF\t'
        for header in self.headers:
            line += header + '\t'
        line += '\n'
        for gene in self.genes:
            line += gene + '\t'
            values = self.genes[gene]
            for value in values:
                line += format(value, '5.3f') + '\t'
            line += '\n'
        line += '!dataset_table_end\n'
    def writeGeoFile(self, filename):
        fh = open(filename, 'w')
        fh.write(str(self))
        fh.close()
        
def readGeoFile(filename, id_column = 0):
    """ Read a Gene Expression Omnibus SOFT file. """
    dataset = None
    fh = open(filename, "r")
    manylines = fh.read()
    fh.close()
    data_rows = False  # Indicates whether we're reading the data section or metadata
    name = u'Unknown'
    cnt_data = 0
    for line in manylines.splitlines():
        if line.startswith('^DATASET'):
            name = line.split('= ')[1]
            continue
        data_rows = line.startswith('!dataset_table_begin')
        data_rows = not line.startswith('!dataset_table_end')
        if len(line.strip()) == 0 or line.startswith('!') or line.startswith('#') or line.startswith('^'):
            continue
        if data_rows:
            cnt_data += 1
            if (cnt_data == 1):  # First line contains the headers
                headers = line.split('\t')
                dataset = GeneProfile(name, headers[2:])  # Create the data set
                continue
            ignore = (dataset == None)  # ignore the row if the dataset is not initialised
            id = line.split('\t')[id_column]
            values = []
            cnt_word = 0
            for word in line.split('\t'):
                cnt_word += 1
                if cnt_word <= (id_column + 1): # ignore the gene names
                    continue
                if word == 'null':
                    ignore = True # ignore gene if a value is null
                    continue
                try:
                    values.append(float(word))
                except:  # ignore values that are not "float"
                    continue
            if not ignore:
                dataset[id] = tuple(values)
    print('Data set %s contains %d genes' % (name, len(dataset.genes)))
    return dataset

With this code in place, you can read a GEO file that you perhaps downloaded from NCBI’s Gene Expression Omnibus (http://www.ncbi.nlm.nih.gov/geo/).